<br>
<center><h1>Tark otsing Riigi Teataja dokumentidel</h1></center>
<br>
<center><h3>Sven Laur* & Tarmo Vaino</h3> </center>

# Ettekande plaan

* Peamised probleemid otsinguga üle eestikeelsest tekstide
* Kuidas antud projekt neid probleeme lahendada püüab
* Lihtsad töövahendid otsingu parandamiseks
* Miks lihtsad lahendused ei tööta
* Eelarvutustel põhinev lahendus 

In [1]:
import math
import requests

from datetime import date
from pandas import DataFrame 
from pandas import read_csv
from pandas import concat
from pandas import merge
from io import StringIO

In [2]:
def search_rt_caption(wordform:str, match_type:str='koik_sonad', **kwargs):
    """
    Potential match types are: 
    * autocomplete – leiab aktid, mille pealkirjas on suvalises järjekorras sõnad, mis algavad otsitud sõnadega
    * koik_sonad – leiab aktid, mille pealkirjas esinevad kõik otsitud sõnad suvalises järjekorras
    * täpne – leiab aktid, mille pealkirjas esineb otsitud fraas
    """
    
    SEARCH_QUERY = 'https://www.riigiteataja.ee/api/oigusakt_otsing/1/otsi'

    ARG_STRUCTURE = \
    {
        'leht': (int, 1),
        'limiit': (int, 500),
        'kehtiv': (date, None),
        'tulemused': (bool, True),
        'kehtivKehtetus': (bool, False),
        'mitteJoustunud': (bool, False),
        'kov': (bool, False),
        'dokument': (str, 'seadus')
    }

    # Check that keyword arguments are correct and have right type 
    payload = kwargs.copy() 
    for key, value in payload.items():
        arg_type, default_value = ARG_STRUCTURE.get(key, (None, None))
        if arg_type is None:
            raise ValueError(f'Unknown argument: {key}')
        elif not isinstance(value, arg_type):
            raise ValueError(f'Argument {key} must be of type {arg_type}')

    # Update important payload arguments        
    payload['leht'] = 1
    payload['kehtiv'] = date.today()
    payload['pealkiri'] = wordform
    payload['pealkiriOtsinguTyyp'] = match_type
    payload['filter'] = True
    payload['grupeeri'] = False
    payload['limiit'] = payload.get('limiit', 500)
    payload['dokument'] = payload.get('dokument', 'seadus')
        
    response = requests.get(SEARCH_QUERY, params=payload)
    assert response.status_code== 200, 'GET request failed'
    response = response.json()

    # Get the number of responce pages
    assert 'aktid' in response, 'Missing payload'
    assert 'metaandmed' in response, 'Missing meta field'
    assert 'kokku' in response['metaandmed'], 'Missing meta field'
    assert 'limiit' in response['metaandmed'], 'Missing meta field'

    total_count = response['metaandmed']['kokku']
    document_limit = response['metaandmed']['limiit']
    max_page = math.ceil(total_count/document_limit)

    if total_count == 0:
        return DataFrame(columns=['document_type', 'document_title', 'commencement_date', 'global_id'])
    
    # Iterate over responce sheets
    query_results = [None] * max_page 
    for page in range(max_page):

        payload['leht'] = page + 1
        response = requests.get(SEARCH_QUERY, params=payload)
        assert response.status_code== 200, 'GET request failed'
        response = response.json()

        document_count = len(response['aktid'])
        query_results[page] = DataFrame({
            'document_type': [None] * document_count,             
            'document_title': [None] * document_count, 
            'commencement_date': [None] * document_count,
            'global_id': [None] * document_count})

        for i, document in enumerate(response['aktid']):
            query_results[page].loc[i, 'global_id'] = document['globaalID']
            query_results[page].loc[i, 'document_title'] = document['pealkiri']
            query_results[page].loc[i, 'document_type'] = document['liik']
            query_results[page].loc[i, 'commencement_date'] = document['kehtivus'].get('algus')
        
    return (concat(query_results, axis=0)
            .sort_values(['document_title', 'commencement_date'], ascending=[True, False])
            .reset_index(drop=True))

search_rt_caption('presidendi')

,document_type,document_title,commencement_date,global_id
0,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2014-01-01,129122012010
1,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2013-01-01,128122011068
2,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2012-01-01,13262582
3,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,122062016013
4,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,127062017010
5,seadus,Vabariigi Presidendi ametihüve seadus,2016-07-01,113122014059
6,seadus,Vabariigi Presidendi ametihüve seadus,2014-01-01,129122011152
7,seadus,Vabariigi Presidendi ametihüve seadus,2012-01-01,128122010026
8,seadus,Vabariigi Presidendi ja Vabariigi Valitsuse li...,2004-07-01,769953
9,seadus,Vabariigi Presidendi töökorra seadus,2017-07-07,127062017011


# Probleem 1

Riigi Teataja otsingu järgi ei vasta otsisõnale **president** ühtegi seadust
* RT otsing ei saa hakkama käändevormidega
* Muutus president → presidendi pole isegi alamsõne  

In [3]:
search_rt_caption('president', match_type='koik_sonad')

,document_type,document_title,commencement_date,global_id


In [4]:
search_rt_caption('president', match_type='autocomplete')

,document_type,document_title,commencement_date,global_id


In [5]:
search_rt_caption('presidendi', match_type='autocomplete')

,document_type,document_title,commencement_date,global_id
0,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2014-01-01,129122012010
1,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2013-01-01,128122011068
2,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2012-01-01,13262582
3,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,122062016013
4,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,127062017010
5,seadus,Vabariigi Presidendi ametihüve seadus,2016-07-01,113122014059
6,seadus,Vabariigi Presidendi ametihüve seadus,2014-01-01,129122011152
7,seadus,Vabariigi Presidendi ametihüve seadus,2012-01-01,128122010026
8,seadus,Vabariigi Presidendi ja Vabariigi Valitsuse li...,2004-07-01,769953
9,seadus,Vabariigi Presidendi töökorra seadus,2017-07-07,127062017011


# Probleem 2

Riigi Teataja laiendatud otsingu järgi vastab otsisõnale **residendi** mitu seadust
* Ükski neist pole seotud residentidega
* Valepositiivsed tulevad selles et residendi → presidendi on alamsõne  

In [6]:
search_rt_caption('residendi', match_type='autocomplete')

,document_type,document_title,commencement_date,global_id
0,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2014-01-01,129122012010
1,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2013-01-01,128122011068
2,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2012-01-01,13262582
3,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,122062016013
4,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,127062017010
5,seadus,Vabariigi Presidendi ametihüve seadus,2016-07-01,113122014059
6,seadus,Vabariigi Presidendi ametihüve seadus,2014-01-01,129122011152
7,seadus,Vabariigi Presidendi ametihüve seadus,2012-01-01,128122010026
8,seadus,Vabariigi Presidendi ja Vabariigi Valitsuse li...,2004-07-01,769953
9,seadus,Vabariigi Presidendi töökorra seadus,2017-07-07,127062017011


In [7]:
search_rt_caption('residendi', match_type='koik_sonad')

,document_type,document_title,commencement_date,global_id


# Probleem 3
Riigi Teataja laiendatud otsingu ei leia ülesse **Strasbourgi** otsisõna
* Kasutaja peab väga täpselt teadma kuidas kohanimi on käänatud
* Laiendatud otsing välistab Strasbourgi alternatiivsed nimekujud Strasbourg'i ja Strasbourg-i

In [8]:
search_rt_caption('Strasbourgi', match_type='autocomplete')

,document_type,document_title,commencement_date,global_id


In [9]:
search_rt_caption("Strasbourg'i", match_type='autocomplete')

,document_type,document_title,commencement_date,global_id
0,seadus,Rahvusvahelise patendiklassifikatsiooni Strasb...,2002-06-01,25055


# Esialgne projektiplaan

<img src="illustrations/summary.png">

# Universaalne teksti normaliseerija 

### I. Lemmatiseerija
* Leiab sõnavormile vastavad algvormid. Neid võib olla mitu
* Veebiteenus https://smart-search.tartunlp.ai/api/analyser/process

### II. Speller
* Leiab valesti kirjutatud sõnavormile kõige lähemad korrektsed vastsed
* Veeniteenus https://smart-search.tartunlp.ai//api/speller/process
* Demorakendus https://smart-search.tartunlp.ai/wp/speller/process


# Lemmatiseerija liidestamine

In [10]:
def analyze_text(text: str):
    
    # Webservice call
    ANALYZER_QUERY = "https://smart-search.tartunlp.ai/api/analyser/process"
    HEADERS = {"Content-Type": "application/json; charset=utf-8"}
    POST_DATA_TEMPLATE = {'params': {"vmetajson": ["--guess"]}, 'content': text}
    response = requests.post(ANALYZER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"
    response = response.json()
    
    # Output reshaping
    token_count = len(response['annotations']['tokens'])
    tbl = DataFrame({'wordform': [None] * token_count, 'lemma': [None] * token_count})
    for i, token in enumerate(response['annotations']['tokens']):
        features = token['features']
        tbl.loc[i, 'wordform'] = features['token']
        tbl.loc[i, 'lemma'] = list(set(map(lambda x: x['lemma'], features['mrf'])))
    tbl =  tbl.reset_index().explode('lemma')

    # Post-correction for Vabamorph output. Remove special symbols 
    tbl['lemma'] = tbl['lemma'].str.replace('=', '', regex=False)
    tbl['lemma'] = tbl['lemma'].str.replace('+', '', regex=False)
    # Post-correction for sublemmas
    tbl['sublemmas'] = tbl['lemma'].str.split('_', regex=False)
    tbl['lemma'] = tbl['lemma'].str.replace('_', '', regex=False)
    return tbl

In [11]:
analyze_text('presidendi')

,index,wordform,lemma,sublemmas
0,0,presidendi,president,[president]


In [12]:
analyze_text('tuumarelvakatsetuste')

,index,wordform,lemma,sublemmas
0,0,tuumarelvakatsetuste,tuumarelvakatsetus,"[tuuma, relva, katsetus]"


In [13]:
analyze_text('sadama')

,index,wordform,lemma,sublemmas
0,0,sadama,sada,[sada]
0,0,sadama,sadam,[sadam]


In [14]:
analyze_text('bresidentide')

,index,wordform,lemma,sublemmas
0,0,bresidentide,bresidenti,[bresidenti]
0,0,bresidentide,bresident,[bresident]


# Spelleri liidestamine

In [15]:
def spell_text(text: str):

    # Webservice call
    SPELLER_QUERY = "https://smart-search.tartunlp.ai/api/speller/process"
    HEADERS = {"Content-Type": "application/json; charset=utf-8"}
    POST_DATA_TEMPLATE = {"content": text}
    response = requests.post(SPELLER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"
    response = response.json()
    
    # Output reshaping
    token_count = len(response['annotations']['tokens'])
    tbl = DataFrame({'wordform': [None] * token_count, 'suggestion': [None] * token_count})
    for i, token in enumerate(response['annotations']['tokens']):
        features = token['features']
        tbl.loc[i, 'wordform'] = features['token']
        tbl.loc[i, 'suggestion'] = features.get('suggestions', features['token'])

    return tbl.reset_index().explode('suggestion')

In [16]:
spell_text('arhidekt')

,index,wordform,suggestion
0,0,arhidekt,arhitekt


In [17]:
spell_text('bresident')

,index,wordform,suggestion
0,0,bresident,resident
0,0,bresident,president


In [18]:
spell_text('aksu')

,index,wordform,suggestion
0,0,aksu,kasu
0,0,aksu,akus
0,0,aksu,asu
0,0,aksu,aku
0,0,aksu,aisu
0,0,aksu,Aksi
0,0,aksu,paksu
0,0,aksu,saksu
0,0,aksu,vaksu
0,0,aksu,laksu


# Universaalne päringu laiendaja

### III. Vormigeneraator
* Leiab algvormis antud sõnast kõikvõimalikud sõnavormid
* Veeniteenus https://smart-search.tartunlp.ai/api/generator/process


In [19]:
def generate_all_worldforms(lemma: str):
    
    # Webservice call
    GENERATOR_QUERY = "https://smart-search.tartunlp.ai/api/generator/process"
    HEADERS = {"Content-Type": "application/json; charset=utf-8"}
    POST_DATA_TEMPLATE = {'type': 'text', 'content': lemma}
    response = requests.post(GENERATOR_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"
    response = response.json()
    assert response['response']['type'] == 'texts', "Unexpected response type"

    # Output reshaping
    token_count = len(response['response']['texts'])
    tbl = DataFrame({'lemma': [None] * token_count, 'wordform': [None] * token_count})
    for i, token in enumerate(response['response']['texts']):
        generated_ = token['features']
        tbl.loc[i, 'lemma'] = token['content']
        tbl.loc[i, 'wordform'] = list(set(map(lambda x: x['token'].replace('+', ''), token['features']['generated_forms'])))

    return tbl.explode('wordform').reset_index(drop=True)

In [20]:
tbl = generate_all_worldforms('president')
concat([tbl[:12], tbl[12:24].reset_index(drop=True), tbl[24:].reset_index(drop=True)], axis=1)

,lemma,wordform,lemma,wordform,lemma,wordform
0,president,presidentisid,president,presidendes,president,presidentidena
1,president,presidente,president,presidentideni,president,presidenti
2,president,presidendel,president,presidentideks,president,presidentide
3,president,presidendina,president,presidendile,president,presidendest
4,president,presidendini,president,presidentidelt,president,presidendesse
5,president,presidendilt,president,presidendil,president,presidentidesse
6,president,presidendisse,president,presidentides,president,presidentideta
7,president,presidendis,president,presidentidest,president,presidendi
8,president,presidentidega,president,presidentidel,president,presidendist
9,president,presidendiks,president,presidendeks,president,presidendid


In [21]:
generate_all_worldforms('Strasbourg')

,lemma,wordform
0,Strasbourg,Strasbourgil
1,Strasbourg,Strasbourgisse
2,Strasbourg,Strasbourgini
3,Strasbourg,Strasbourgiks
4,Strasbourg,Strasbourgilt
5,Strasbourg,Strasbourgina
6,Strasbourg,Strasbourgi
7,Strasbourg,Strasbourgile
8,Strasbourg,Strasbourgis
9,Strasbourg,Strasbourgita


# Lihtne pealkirjaotsing üle Riigi Teataja seaduste 

1. Spelli sisend
2. Leia sisendile vastavad lemmad
3. Genereeri kõik sõnavormid
4. Kombineeri saadud päringute vastused

In [22]:
def smart_search_rt_caption(wordform:str, match_type:str='koik_sonad', **kwargs):

    # Correct and extend search query 
    search_words = spell_text(wordform)['suggestion']
    search_lemmas = concat([analyze_text(word)['lemma'] for word in search_words], axis=0)
    search_wordforms = concat([generate_all_worldforms(lemma)['wordform'] for lemma in search_lemmas], axis=0)

    # Interact with RT webservice
    result = [None] * len(search_wordforms)
    for i, wordform in enumerate(search_wordforms): 
        print('#', end='')
        result[i] = search_rt_caption(wordform, match_type=match_type, **kwargs)
        if i % 40 == 19:
            print(' ', end="")
        elif  i % 40 == 39:
            print()
    
    # Reshape the outcome
    return (concat(result, axis=0)
            .sort_values(['document_title', 'commencement_date'], ascending=[True, False])
            .reset_index(drop=True))   

In [23]:
smart_search_rt_caption('bresidenti')

#################### ####################
#################### ############

,document_type,document_title,commencement_date,global_id
0,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2014-01-01,129122012010
1,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2013-01-01,128122011068
2,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2012-01-01,13262582
3,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,122062016013
4,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,127062017010
5,seadus,Vabariigi Presidendi ametihüve seadus,2016-07-01,113122014059
6,seadus,Vabariigi Presidendi ametihüve seadus,2014-01-01,129122011152
7,seadus,Vabariigi Presidendi ametihüve seadus,2012-01-01,128122010026
8,seadus,Vabariigi Presidendi ja Vabariigi Valitsuse li...,2004-07-01,769953
9,seadus,Vabariigi Presidendi töökorra seadus,2017-07-07,127062017011


In [24]:
smart_search_rt_caption('aksu')

#################### ####################
#################### ####################
#################### ####################
#################### ####################
#################### ####################
#################### ####################
#################### ####################
#################### ####################
#################### ####################
#################### ####################
#################### ####################
#################### ####################
#################### ###

,document_type,document_title,commencement_date,global_id
0,seadus,Eesti Vabariigi Valitsuse ning Suurbritannia j...,2002-06-01,24794
1,seadus,Eesti Vabariigi Valitsuse ning Suurbritannia j...,2002-06-01,24794
2,seadus,Eesti Vabariigi ja Ameerika Ühendriikide vahel...,2002-06-01,79003
3,seadus,Eesti Vabariigi ja Araabia Ühendemiraatide vah...,2012-03-16,206032012003
4,seadus,Eesti Vabariigi ja Armeenia Vabariigi vahelise...,2003-01-20,237128
...,...,...,...,...
76,seadus,Kohalike maksude seadus,2012-01-01,113122011004
77,seadus,Kohalike maksude seadus,2012-01-01,13361798
78,seadus,Makse- ja arveldussüsteemide seadus,2023-03-11,101032023003
79,seadus,Riiklike maksude intresside kustutamise seadus,2014-04-26,116042014079


# Ettearvutused päringu laiendamiseks
<img src="illustrations/eelarvutused_päringu_laiendamiseks.png">

# Probleemid sisendtekstidega

* Äriseadustik1
* Nõuded lõppseadmele1
* Kasvatusraskustega 7  .  ¿9  .   klassi õpilaste klassi moodustamise tingimuste ja korra kinnitamine
* «Maareformi seaduse» paragrahvides 23¹ ja 23² sätestatud vaba põllumajandus- ja metsamaa erastamise kord

**Märkus:** Täistekstide analüüs on veelgi keerulisem
* Esmalt on vaja XML sisendist eraldada tekst
* Teiseks on vaja tekstist eraldada tekstivälised osad
* Ideaalis on tarvis taastada lõikudele vastav vorminamata tekst


# Indekseerimine

### IV. Sõnavormide indekseerija
* Leiab sõnavormile vastavad algvormid. Neid võib olla mitu.
* Veebiteenus https://smart-search.tartunlp.ai/api/sonede-indekseerija/
* Demorakendus https://smart-search.tartunlp.ai/wp/indekseerija/process

### V. Algvormide indekseerija
* Leiab sõnavormile vastavad algvormid. Neid võib olla mitu.
* Veebiteenus https://smart-search.tartunlp.ai/api/lemmade-indekseerija/
* Demorakendus https://smart-search.tartunlp.ai/api/sonede-indekseerija/


In [25]:
def index_worldforms(text: str):
    
    # Webservice call
    INDEXER_QUERY = "https://smart-search.tartunlp.ai/api/sonede-indekseerija/csv"
    HEADERS = {"Content-Type": "application/json; charset=utf-8"}
    POST_DATA_TEMPLATE = {"sources": {'xxx':{"content": text}}}
    response = requests.post(INDEXER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"

    # Output reshaping
    tbl = read_csv(StringIO(response.text), delimiter='\t', header=None)
    tbl = tbl.set_axis(['wordfrorm', 'is_subword', 'word','document', 'start', 'end'], axis=1)
    return tbl[['wordfrorm', 'is_subword', 'word', 'start', 'end' ]].sort_values('start')


In [26]:
index_worldforms('Kui Arno isaga külakooli jõudis olid tunnid juba alanud')

,wordfrorm,is_subword,word,start,end
7,kui,False,Kui,0,3
0,Arno,False,Arno,4,8
3,isaga,False,isaga,9,14
6,kooli,True,külakooli,15,24
8,küla,True,külakooli,15,24
9,külakooli,False,külakooli,15,24
5,jõudis,False,jõudis,25,31
10,olid,False,olid,32,36
11,tunnid,False,tunnid,37,43
4,juba,False,juba,44,48


In [27]:
def index_lemmas(text: str):
    
    # Webservice call
    INDEXER_QUERY = "https://smart-search.tartunlp.ai/api/lemmade-indekseerija/csv"
    HEADERS = {"Content-Type": "application/json; charset=utf-8"}
    POST_DATA_TEMPLATE = {"sources": {'xxx':{"content": text}}}
    response = requests.post(INDEXER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"
    
    # Output reshaping
    tbl = read_csv(StringIO(response.text), delimiter='\t', header=None)
    tbl = tbl.set_axis(['lemma', 'is_subword', 'word','document', 'start', 'end'], axis=1)
    return tbl[['lemma', 'is_subword', 'word', 'start', 'end' ]].sort_values('start')

In [28]:
index_lemmas('Kui Arno isaga külakooli jõudis olid tunnid juba alanud')

,lemma,is_subword,word,start,end
8,kui,False,Kui,0,3
0,Arno,False,Arno,4,8
4,isa,False,isaga,9,14
7,kool,True,külakooli,15,24
9,küla,True,külakooli,15,24
10,külakool,False,külakooli,15,24
6,jõudma,False,jõudis,25,31
11,olema,False,olid,32,36
12,tund,False,tunnid,37,43
5,juba,False,juba,44,48


# Lõpptabelid

# Sõnavormide sagedusloend 

|wordform|occurence_count|document_count|
|:---|---:|---:|
|seadus           |5950 |2975 |
|ja               |1178 | 918 |
|Eesti            | 726 | 324 |
|Vabariigi        | 577 | 337 |
|ratifitseerimise | 472 | 469 |
|Euroopa          | 442 | 174 |
|seadustik        | 249 | 249 |
|vahelise         | 218 | 204 |
|kaitse           | 210 | 105 |

# Algvormide sagedusloend 

|sublemma|lemma|occurence_count|document_count|
|:---|:---|---:|---:|
|False | seadus           | 3005 | 2989 | 
|False | seadu            | 2989 | 2989 |
|False | ja               | 1178 |  918 |
|True  | vaba             |  616 |  338 |
|False | vabariik         |  584 |  338 |
|False | ratifitseerimine |  472 |  469 |
|True  | maksu            |  374 |  373 |
|False | Eesti            |  370 |  330 |
|False | eesti            |  363 |  324 | 


# Kirjavigate korrigeerimisõnastik

|misspelling | search_string |lemma|
|:---|:---|:---|
|True |Enamsoodustussrežiimideks | Enamsoodustusrežiim
|True |Enamsoodustussrežiimidel  | Enamsoodustusrežiim
|True |Enamsoodustussrežiimidele | Enamsoodustusrežiim
|True |Enamsoodustussrežiimidelt | Enamsoodustusrežiim
|True |Enamsoodustussrežiimidena | Enamsoodustusrežiim

# Eelarvutamisega otsing üle pealkirjade 


### VI. Eelarvutamisega päringulaiendaja
* Leiab otsisõnale vastavad algvormid, mis esinevad tekstis 
* Veebiteenus https://smart-search.tartunlp.ai/api/ea_paring/json
* Demorakendus https://smart-search.tartunlp.ai/wp/ea_paring_otsing/process


In [29]:
def extend_search_query(search_input: str):
    
    # Webservice call
    SEARCH_LEMMATISATION_QUERY = 'https://smart-search.tartunlp.ai/api/ea_paring/json' 
    HEADERS = {'Content-Type': 'application/json'}
    POST_DATA_TEMPLATE ={"content": search_input}
    response = requests.post(SEARCH_LEMMATISATION_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"
    response = response.json()
    
    # Output reshaping
    assert 'annotations' in response, "Invalid response"
    assert 'query' in response['annotations'], "Invalid response"
    assert 'typos' in response['annotations'], "Invalid response"

    query = response['annotations']['query']
    typos = response['annotations']['typos'].get(search_input)
    
    tbl = DataFrame({'input': [None], 'lemma': [None], 'suggestion': [None]})
    tbl.loc[0, 'input'] = search_input
    tbl.loc[0, 'lemma'] = query[0] if len(query) != 0 else None
    tbl.loc[0, 'suggestion'] = [element['suggestion'] for element in typos] if typos is not None else None 

    return tbl.explode(['lemma', 'suggestion'])                    

In [30]:
extend_search_query('president')

,input,lemma,suggestion
0,president,president,None


In [31]:
extend_search_query('bresident')

,input,lemma,suggestion
0,bresident,None,president


In [32]:
extend_search_query('xxxx')

,input,lemma,suggestion
0,xxxx,None,None


# Mis on veel pooleli... 

* Dokumenteeritud töövood Riigi Teataja pealkirja- ja sisuotsinguks
  * Eeltöötlus
  * Näiteliidestus RT veeniteenusega 
  * Uuendamist käsitlev osa

* Mõningate veebiteenuste sisend- ja väljundformaat
  * Optimeerida esitust
  * Täiendada dokumentatsiooni

* Osasõnade korrektne käsitlus päringulaiendajas
  * RT veeviteenus ei oska osasõnu ja suvalisi teksiosasid eraldada

* Riigiteataja tekstide süvaanalüüs
  * Riikide, asutuste ja isikute tuvastamine